# P1: Trinomial Tree
In the lecture we mentioned that a binomial model can be generalized to a trinomial model. In a trinomial model we need to consider three stock price developments: in one period the price increases by a factor of $u$ with the probability $p_u$, descreases by a factor of $d$ with the probability $p_d$, or remains unchanged with the probability $1−p_u −p_d$. The trinomial model can be built in a similar way to the binomial model, e.g., by matching the expectation and the variance and using $ud = 1$ we obtain:
    $$ u = e^{\sigma\sqrt{2\Delta t}}, \quad d = e^{-\sigma\sqrt{2\Delta t}}, \quad p_d = \left( \frac{ e^{\sigma\sqrt{\frac{\Delta t}{2}}} - e^{r\frac{\Delta t}{2}} }{e^{\sigma\sqrt{\frac{\Delta t}{2}}} - e^{-\sigma\sqrt{\frac{\Delta t}{2}}}} \right)^2, \quad \quad p_u = \left( \frac{ e^{r\frac{\Delta t}{2}} - e^{-\sigma\sqrt{\frac{\Delta t}{2}}}}{e^{\sigma\sqrt{\frac{\Delta t}{2}}} - e^{-\sigma\sqrt{\frac{\Delta t}{2}}}} \right)^2.$$

For the programming task write the corresponding Python functions that are asked in each section below.

**Return the solutions of the programming task until Friday, May 16. The assignment can be done in groups of 1-2 persons. For the presentation of your solution, please register via the moodle lab course.**

## Write your data

In [1]:
print("First Name: Hamidreza")
print("Sure Name: Lolaei")
print("Matriculation number: 2225940")
print("E-mail: hamidreza.lolaei@uni-wuppertal.de")

First Name: Hamidreza
Sure Name: Lolaei
Matriculation number: 2225940
E-mail: hamidreza.lolaei@uni-wuppertal.de


## Import all necessary modules

In [2]:
import math
import numpy as np
from tabulate import tabulate
np.set_printoptions(linewidth=600)

## Test Parameters

In [3]:
# Input parameters
sigma = 0.2
K = 100
S0 = 100
N = 4
r = 0.05
T = 1
d_t = T / N

# Start coding
**Note:** 
* Add code where you find the comment "# Add code here"
* Get the same output as it is presented in the Markdown text **Expected Output:**

## Define a function to calculate the up and down factors

In [4]:
def calc_ud(sigma, d_t):
    """
    Function that calculates the factors in the trinomial model

    Input: sigma -> volatility 
            d_t  -> time step size

    Output: u -> upward movement
            d -> downward movement

    """
    # Add code here
    u = math.exp(sigma*math.sqrt(2*d_t))
    d = math.exp(-sigma*math.sqrt(2*d_t))

    return u , d

In [5]:
# Calculate upward and downward movements
# Add code here
u , d = calc_ud(sigma , d_t)
u = np.round(u , decimals = 6)
d = np.round(d , decimals = 6)
print(u)
print(d)

1.15191
0.868123


**Expected Output:**
`u = 1.151910, d = 0.868123`

## Define a function to calculate the probabilities in the trinomial model

In [6]:
def calc_prob(r, sigma, d_t):
    """
    Function that calculates the probabilities in the trinomial model

    Input: r     -> interest rate
           sigma -> volatility
           d_t   -> time step size

    Output: p_u -> probability for upward movement
            p_d -> probability for downward movement
            p_m -> probability for no change

    """
    s1 = math.exp(sigma*math.sqrt(d_t/2))
    s2 = math.exp(-sigma*math.sqrt(d_t/2))
    s3 = math.exp(r*d_t/2)
    
    p_d = ((s1 - s3)/(s1 - s2))**2
    p_u = ((s3 - s2)/(s1 - s2))**2
    p_m = 1 - p_u - p_d

    return p_u , p_d , p_m
    
    

In [7]:
# Calculate probabilities
# Add code here
p_u , p_d , p_m = calc_prob(r , sigma , d_t)
p_u = np.round(p_u , decimals = 6)
p_d = np.round(p_d , decimals = 6)
p_m = np.round(p_m , decimals = 6)
print('p_u = ' ,p_u)
print('p_d = ' ,p_d)
print('p_m = ' ,p_m)

p_u =  0.277334
p_d =  0.224084
p_m =  0.498582


**Expected Output:**
`p_u = 0.277334, p_d = 0.224084, p_m = 0.498582`

## Define a function to calculate the stock prices in the trinomial model

In [8]:
def stock_price_trinom(S0, N, u, d):
    
    """
    Function that calculates the stock prices in the trinomial model
    
    Input: S0 -> Initial Stock Price
           u  -> upward movement
           d  -> downward movement
           N  -> Number of time periods
       
    Output: S -> Upper triangular matrix of Stock prices with tirnomial model
    """
    arr = np.zeros((2*N+1,N+1))
    for col in range(N + 1):
        for i in range(2*col + 1):
            j = max(0 , i - col)
            k = max(col - i, 0)
            arr[i   , col] = S0*(u**(k))*(d**(j))
    return arr    


In [9]:
# Calculate stock prices
# Add code here
S = stock_price_trinom(S0, N, u, d)
print(S)

[[100.         115.191      132.68966481 152.84655179 176.06547147]
 [  0.         100.         115.191      132.68966481 152.84655179]
 [  0.          86.8123     100.         115.191      132.68966481]
 [  0.           0.          86.8123     100.         115.191     ]
 [  0.           0.          75.36375431  86.8123     100.        ]
 [  0.           0.           0.          75.36375431  86.8123    ]
 [  0.           0.           0.          65.42500849  75.36375431]
 [  0.           0.           0.           0.          65.42500849]
 [  0.           0.           0.           0.          56.79695464]]


**Expected Output:**

`Stock Prices`

`[[100.         115.19099102 132.68964411 152.84651603 176.06541655]
 [  0.         100.         115.19099102 132.68964411 152.84651603]
 [  0.          86.81234454 100.         115.19099102 132.68964411]
 [  0.           0.          86.81234454 100.         115.19099102]
 [  0.           0.          75.36383164  86.81234454 100.        ]
 [  0.           0.           0.          75.36383164  86.81234454]
 [  0.           0.           0.          65.42510919  75.36383164]
 [  0.           0.           0.           0.          65.42510919]
 [  0.           0.           0.           0.          56.7970712 ]]`

## Define a function to calculate the European option price using trinomial model

In [10]:
def priceTriEuro(N, K, r, S, p_u, p_d, p_m, d_t):
    """
    Function that calculates the European option price using trinomial model
    
    Input: N     -> Number of time periods
           K     -> Strike price
           r     -> Interst rate
           S     -> Upper triangular matrix
           p_u   -> probability for upward movement
           p_d   -> probability for downward movement
           p_m   -> probability for no change
           d_t   -> time step size
       
    Output: C -> European Call option 
            P -> European Put option 

    """
    # Add code here
    # European call option matrix
    C = np.zeros((2*N+1 , N+1))
    P = np.zeros((2*N+1 , N+1))
    C[: , N] = np.maximum(S[: , N] - K , 0) # payoff functions in the last step
    P[: , N] = np.maximum(K - S[: , N] , 0) # payoff functions in the last step
    for i in range(N - 1  , -1, -1):
        for j in range(2*i + 1):
            C[j , i] = math.exp(-r*d_t)*(p_u*C[j , i + 1] + p_m*C[j + 1 , i+1] + p_d*C[j+2 , i+ 1])
            P[j , i] = math.exp(-r*d_t)*(p_u*P[j , i+ 1] + p_m*P[j + 1 , i+ 1] + p_d*P[j+2 , i+ 1])
    return C , P , C[0 , 0] , P[0 , 0]

         

In [11]:
# Calculate European call and put values
# Add code here
C , P, _ , _ = priceTriEuro(N, K, r, S, p_u, p_d, p_m, d_t)
print('C = \n' , C) 
print('\n\nP = \n\n', P)

C = 
 [[10.20508408 20.09256307 35.15865143 54.08875897 76.06547147]
 [ 0.          8.49086296 18.30584026 33.93187367 52.84655179]
 [ 0.          2.35494685  6.54952838 16.43321032 32.68966481]
 [ 0.          0.          1.13955485  4.16064631 15.191     ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


P = 

 [[ 5.32807446  1.22103717  0.          0.          0.        ]
 [ 0.          4.81034308  0.64585257  0.          0.        ]
 [ 0.         11.86210434  4.08054774  0.          0.        ]
 [ 0.          0.         11.858255    2.91844307  0.        ]
 [ 0.          0.         22.1672414  11.94548265  0.        ]
 [ 0.          0.          0.         23.39402799 13.1877    ]
 [ 0.          0.          0.         3

**Expected Output:**

`European Call`

`[[10.20509942 20.0925775  35.15865291 54.08873598 76.06541655]
 [ 0.          8.49087322 18.30584772 33.93186407 52.84651603]
 [ 0.          2.35495086  6.54953395 16.43321097 32.68964411]
 [ 0.          0.          1.13955621  4.16064756 15.19099102]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]`

`European Put`

`[[ 5.32804187  1.22102826  0.          0.          0.        ]
 [ 0.          4.810315    0.64584791  0.          0.        ]
 [ 0.         11.86204809  4.08052515  0.          0.        ]
 [ 0.          0.         11.85820287  2.91842761  0.        ]
 [ 0.          0.         22.16715956 11.94543551  0.        ]
 [ 0.          0.          0.         23.39394841 13.18765546]
 [ 0.          0.          0.         33.33267086 24.63616836]
 [ 0.          0.          0.          0.         34.57489081]
 [ 0.          0.          0.          0.         43.2029288 ]]`

## Define a function to calculate the American option price using trinomial model

In [12]:
def priceTriAmer(N, K, r, S, p_u, p_d, p_m, d_t):
    """
    Function that calculates the American option price using trinomial model
    
    Input: N     -> Number of time periods
           K     -> Strike price
           r     -> Interst rate
           S     -> Upper triangular matrix
           p_u   -> probability for upward movement
           p_d   -> probability for downward movement
           p_m   -> probability for no change
           d_t   -> time step size
       
    Output: C -> American Call option 
            P -> American Put option 

    """
    C = np.zeros((2*N+1 , N+1))
    P = np.zeros((2*N+1 , N+1))
    C[: , N] = np.maximum(S[: , N] - K , 0)
    P[: , N] = np.maximum(K - S[: , N] , 0)
    for i in range(N - 1  , -1, -1):
        for j in range(2*i + 1):
            C_temp = math.exp(-r*d_t)*(p_u*C[j , i + 1] + p_m*C[j + 1 , i+1] + p_d*C[j+2 , i+ 1])
            C[j , i] = np.maximum(S[j , i] - K , C_temp)
            P_temp = math.exp(-r*d_t)*(p_u*P[j , i+ 1] + p_m*P[j + 1 , i+ 1] + p_d*P[j+2 , i+ 1])
            P[j , i] = np.maximum(K  - S[j , i], P_temp)
    return C , P , C[0 , 0] , P[0 , 0]

In [13]:
# Calculate American call and put values
# Add code here
CA , PA , _ , _ = priceTriAmer(N, K, r, S, p_u, p_d, p_m, d_t)
print('CA = \n' , CA) 
print('\n\nPA = \n\n', PA)

CA = 
 [[10.20508408 20.09256307 35.15865143 54.08875897 76.06547147]
 [ 0.          8.49086296 18.30584026 33.93187367 52.84655179]
 [ 0.          2.35494685  6.54952838 16.43321032 32.68966481]
 [ 0.          0.          1.13955485  4.16064631 15.191     ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


PA = 

 [[ 5.84960493  1.28187335  0.          0.          0.        ]
 [ 0.          5.23990894  0.64585257  0.          0.        ]
 [ 0.         13.1877      4.35545092  0.          0.        ]
 [ 0.          0.         13.1877      2.91844307  0.        ]
 [ 0.          0.         24.63624569 13.1877      0.        ]
 [ 0.          0.          0.         24.63624569 13.1877    ]
 [ 0.          0.          0.        

**Expected Output:**

`American Call`

`[[10.20509942 20.0925775  35.15865291 54.08873598 76.06541655]
 [ 0.          8.49087322 18.30584772 33.93186407 52.84651603]
 [ 0.          2.35495086  6.54953395 16.43321097 32.68964411]
 [ 0.          0.          1.13955621  4.16064756 15.19099102]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]`

`American Put`

`[[ 5.84957502  1.28186433  0.          0.          0.        ]
 [ 0.          5.23988204  0.64584791  0.          0.        ]
 [ 0.         13.18765546  4.35542837  0.          0.        ]
 [ 0.          0.         13.18765546  2.91842761  0.        ]
 [ 0.          0.         24.63616836 13.18765546  0.        ]
 [ 0.          0.          0.         24.63616836 13.18765546]
 [ 0.          0.          0.         34.57489081 24.63616836]
 [ 0.          0.          0.          0.         34.57489081]
 [ 0.          0.          0.          0.         43.2029288 ]]`

## Comparing European and American option values calculated with Trinomial Model from the exact Solution

In [14]:
# Calculate European and American put values for different N and compare to exact solution
# Exact Values
E_P_ex = 5.5735
A_P_ex = 6.0624
N1 = [4, 16, 64, 256, 512, 1024]
# Add code here
data = np.zeros((6 , 7))
ii = 0
for n in N1:
    d_t_new = T/n
    u_new , d_new = calc_ud(sigma, d_t_new)
    p_un, p_dn , p_mn = calc_prob(r, sigma, d_t_new)
    S1 = stock_price_trinom(S0, n, u_new, d_new)
    _ , _ , _ , E_P_app = priceTriEuro(n, K, r, S1, p_un, p_dn, p_mn, d_t_new)
    _, _ , _ , A_P_app = priceTriAmer(n, K, r, S1, p_un, p_dn, p_mn, d_t_new)
    err_E_P = abs(E_P_app - E_P_ex)
    err_A_P = abs(A_P_app - A_P_ex)
    data[ ii , :] = [int(n) , E_P_ex , E_P_app , err_E_P , A_P_ex , A_P_app , err_A_P]
    ii +=1

headers = headers = ["N", "E_P_ex", "E_P_app", "err_E_P" , "A_P_ex" , "A_P_app", "err_A_P"]
print(tabulate(data, headers=headers))

   N    E_P_ex    E_P_app      err_E_P    A_P_ex    A_P_app    err_A_P
----  --------  ---------  -----------  --------  ---------  ---------
   4    5.5735    5.32804  0.245458       6.0624    5.84958  0.212825
  16    5.5735    5.51129  0.0622119      6.0624    6.03113  0.0312749
  64    5.5735    5.55792  0.0155814      6.0624    6.07702  0.0146224
 256    5.5735    5.56962  0.00387872     6.0624    6.08734  0.0249399
 512    5.5735    5.57157  0.00192659     6.0624    6.08891  0.0265064
1024    5.5735    5.57255  0.000950343    6.0624    6.08965  0.0272517


**Expected Output:**
![title](trinom_tree.png)

In [15]:
# feel free to use this cell for additional tests

In [16]:
# feel free to use this cell for additional tests